In [3]:
import matplotlib.pyplot as plt
import cv2
import numpy as np 
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
tf.test.is_gpu_available()

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
def count_files_in_folder(folder_path):
    file_count = len([filename for filename in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, filename))])
    return file_count


fold_names = ['Fold1', 'Fold2', 'Fold3', 'Fold4', 'Fold5']

train_total=0
val_total=0

# 遍历每个Fold文件夹
for fold_name in fold_names:
    train_folder_path = f'./ETT_v3/{fold_name}/train'
    val_folder_path = f'./ETT_v3/{fold_name}/val'
    
    # 获取train文件夹中的文件数量
    num_train_files = count_files_in_folder(train_folder_path)
    train_total=num_train_files+train_total
    print(f"{fold_name} train文件夹中的文件数量：{num_train_files}")
    
    # 获取val文件夹中的文件数量
    num_val_files = count_files_in_folder(val_folder_path)
    val_total=val_total+num_val_files
    print(f"{fold_name} val文件夹中的文件数量：{num_val_files}")

print(train_total)
print(val_total)

In [22]:
def preprocess(imgpath):
    # os.mkdir('%s_New'%(imgpath))
    image_folder = imgpath
    images = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]
    print(images)
    image_list = []
    for img in images:
        # print(Image.open(img).size)
        imgOpen = Image.open(img)
        if imgOpen.mode =='RGB':
            imgOpen = imgOpen.convert('L')
        width,height = imgOpen.size
        print(f"图像 {img} 的尺寸为：{width} x {height}")
        # 定義裁切區域的左上角座標和右下角座標
        left = width/4
        top = 0
        right = width*(3/4)
        bottom = height/2
        # 使用crop()方法裁切圖像
        cropped_img = imgOpen.crop((left, top, right, bottom))
        #將圖像大小調整至256*256
        
        cropped_img_resize = cropped_img.resize((256,256))
        #存入新的資料夾中
        # parts = img.split('\\')#已\\為基準分左右兩部分
        # cropped_img_resize.save('%s_New/%s'%(imgpath, parts[1]))
        image_list.append(cropped_img_resize)
        image_list.append(np.array(cropped_img_resize))
    # image_array = np.array(image_list)
    return np.array(image_list)

In [23]:
images = preprocess('./ETT_v3/Fold1/train')
# images_val = preprocess('./ETT_v3/Fold1/test')

['./ETT_v3/Fold1/train\\A00010.jpg', './ETT_v3/Fold1/train\\A000100.jpg', './ETT_v3/Fold1/train\\A000110.jpg', './ETT_v3/Fold1/train\\A000120.jpg', './ETT_v3/Fold1/train\\A000130.jpg', './ETT_v3/Fold1/train\\A000140.jpg', './ETT_v3/Fold1/train\\A000160.jpg', './ETT_v3/Fold1/train\\A00020.jpg', './ETT_v3/Fold1/train\\A000200.jpg', './ETT_v3/Fold1/train\\A000210.jpg', './ETT_v3/Fold1/train\\A000220.jpg', './ETT_v3/Fold1/train\\A000230.jpg', './ETT_v3/Fold1/train\\A000240.jpg', './ETT_v3/Fold1/train\\A000250.jpg', './ETT_v3/Fold1/train\\A000260.jpg', './ETT_v3/Fold1/train\\A000270.jpg', './ETT_v3/Fold1/train\\A000280.jpg', './ETT_v3/Fold1/train\\A00030.jpg', './ETT_v3/Fold1/train\\A000300.jpg', './ETT_v3/Fold1/train\\A000320.jpg', './ETT_v3/Fold1/train\\A000330.jpg', './ETT_v3/Fold1/train\\A000340.jpg', './ETT_v3/Fold1/train\\A00040.jpg', './ETT_v3/Fold1/train\\A00050.jpg', './ETT_v3/Fold1/train\\A000600.jpg', './ETT_v3/Fold1/train\\A00080.jpg', './ETT_v3/Fold1/train\\A00090.jpg', './ETT_

KeyboardInterrupt: 

In [ ]:
mask = preprocess('./ETT_v3/Fold1/trainannot')
mask_val = preprocess('./ETT_v3/Fold1/testannot')

In [ ]:
# Define U-Net model
def unet_model(input_size=(256, 256, 1)):
    inputs = tf.keras.Input(input_size)
    
    # Contracting Path
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = layers.Dropout(0.5)(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(drop4)
    
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = layers.Dropout(0.5)(conv5)
    
    # Expansive Path
    up6 = layers.Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(layers.UpSampling2D(size=(2, 2))(drop5))
    merge6 = layers.concatenate([drop4, up6], axis=3)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)
    
    up7 = layers.Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(layers.UpSampling2D(size=(2, 2))(conv6))
    merge7 = layers.concatenate([conv3, up7], axis=3)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)
    
    up8 = layers.Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(layers.UpSampling2D(size=(2, 2))(conv7))
    merge8 = layers.concatenate([conv2, up8], axis=3)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)
    
    up9 = layers.Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(layers.UpSampling2D(size=(2, 2))(conv8))
    merge9 = layers.concatenate([conv1, up9], axis=3)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv9 = layers.Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    
    conv10 = layers.Conv2D(1, 1, activation='sigmoid')(conv9)
    
    model = models.Model(inputs=inputs, outputs=conv10)
    return model



In [ ]:
# Build the U-Net model
model = unet_model()

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(images, mask, batch_size=4, epochs=10, validation_data=(images_val, mask_val))

# Save the model
model.save("unet_model.h5")